<a href="https://colab.research.google.com/github/PDGamerSG/GDSC-Image_Classification/blob/main/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pdgamersg","key":"35378d4ad319c63c6a143638f767f63c"}'}

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets list

ref                                                              title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
abdulmalik1518/mobiles-dataset-2025                              Mobiles Dataset (2025)                            20KB  2025-02-18 06:50:24           6184        106  1.0              
asinow/car-price-dataset                                         Car Price Dataset                                135KB  2025-01-26 19:53:28          22998        370  1.0              
mahmoudelhemaly/students-grading-dataset                         Student Performance & Behavior Dataset           508KB  2025-02-17 17:38:46           4980         98  1.0              
samikshadalvi/lungs-diseases-dataset                             Lungs

In [7]:
!kaggle competitions download -c cifar-10

100% 714M/715M [00:07<00:00, 147MB/s]
100% 715M/715M [00:07<00:00, 98.0MB/s]


In [8]:
!unzip cifar-10.zip

Archive:  cifar-10.zip
  inflating: sampleSubmission.csv    
  inflating: test.7z                 
  inflating: train.7z                
  inflating: trainLabels.csv         


In [9]:
!apt-get install p7zip-full
!7z x train.7z

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 109723070 bytes (105 MiB)

Extracting archive: train.7z
--
Path = train.7z
Type = 7z
Physical Size = 109723070
Headers Size = 294768
Method = LZMA:26
Solid = +
Blocks = 1

  0%      2%      2% 669 - train/1060.png                           2% 1162 - train/11043.png                             2% 1496 - train/11344.png                           

In [10]:
!apt-get install p7zip-full
!7z x test.7z

^C

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 639374249 bytes (610 MiB)

Extracting archive: test.7z
--
Path = test.7z
Type = 7z
Physical Size = 639374249
Headers Size = 1721070
Method = LZMA:26
Solid = +
Blocks = 1

  0%    
Archives with Errors: 1



Break signaled


In [11]:
import os
import shutil
import pandas as pd
df = pd.read_csv('trainLabels.csv')
source_folder = 'train'

categories = df['label'].unique()
for category in categories:
    os.makedirs(category, exist_ok=True)

for index, row in df.iterrows():
    filename = f"{row['id']}.png" #f does that same thing only give value and then add at last png to make it a image
    label = row['label']

    source_file = os.path.join(source_folder, filename)
    destination_file = os.path.join(label, filename)

    if os.path.exists(source_file):
        shutil.move(source_file, destination_file) #helps to move the file from the location
        print(f"Moved {filename} to {label} folder")
    else:
        print(f"Warning: File {filename} not found")

print("Image sorting complete")


Streaming output truncated to the last 5000 lines.
Moved 45002.png to automobile folder
Moved 45003.png to deer folder
Moved 45004.png to automobile folder
Moved 45005.png to airplane folder
Moved 45006.png to bird folder
Moved 45007.png to bird folder
Moved 45008.png to dog folder
Moved 45009.png to truck folder
Moved 45010.png to frog folder
Moved 45011.png to airplane folder
Moved 45012.png to automobile folder
Moved 45013.png to cat folder
Moved 45014.png to automobile folder
Moved 45015.png to automobile folder
Moved 45016.png to bird folder
Moved 45017.png to automobile folder
Moved 45018.png to ship folder
Moved 45019.png to dog folder
Moved 45020.png to frog folder
Moved 45021.png to truck folder
Moved 45022.png to bird folder
Moved 45023.png to deer folder
Moved 45024.png to bird folder
Moved 45025.png to ship folder
Moved 45026.png to ship folder
Moved 45027.png to cat folder
Moved 45028.png to horse folder
Moved 45029.png to automobile folder
Moved 45030.png to horse folder


In [13]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
destination_folder = 'All Categories'
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)
    print(f"Folder '{destination_folder}' created successfully.")
folders_move = ['cat', 'dog','airplane','bird','deer','frog','horse','ship','train','truck','automobile']
for folder in folders_move:
    if os.path.exists(folder):
        shutil.move(folder, destination_folder)

class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.classes = sorted(os.listdir(folder_path))
        self.image_paths = []

        for class_index, class_name in enumerate(self.classes):
            class_folder = os.path.join(folder_path, class_name)
            for img_file in os.listdir(class_folder):
                self.image_paths.append((os.path.join(class_folder, img_file), class_index))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        img_path, label = self.image_paths[index]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = ImageDataset(folder_path='/content/All Categories', transform=transform)

data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

print(f"Total images: {len(dataset)}")
print(f"Classes: {dataset.classes}")
print(f"Number of batches: {len(data_loader)}")

for class_name in dataset.classes:
    class_folder = os.path.join('/content/All Categories', class_name)
    num_images = len(os.listdir(class_folder))
    print(f"{class_name}: {num_images} images")

for images, labels in data_loader:
    print(f"Batch shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    break


Folder 'All Categories' created successfully.
Total images: 50000
Classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'train', 'truck']
Number of batches: 782
airplane: 5000 images
automobile: 5000 images
bird: 5000 images
cat: 5000 images
deer: 5000 images
dog: 5000 images
frog: 5000 images
horse: 5000 images
ship: 5000 images
train: 0 images
truck: 5000 images
Batch shape: torch.Size([64, 3, 32, 32])
Labels shape: torch.Size([64])


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

num_classes = len(dataset.classes)
model = SimpleCNN(num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(data_loader):.4f}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the dataset: {100 * correct / total}%')

Epoch [1/10], Loss: 1.5169
Epoch [2/10], Loss: 1.1054
Epoch [3/10], Loss: 0.9240
Epoch [4/10], Loss: 0.8142
Epoch [5/10], Loss: 0.7397
Epoch [6/10], Loss: 0.6776
Epoch [7/10], Loss: 0.6311
Epoch [8/10], Loss: 0.5793
Epoch [9/10], Loss: 0.5438
Epoch [10/10], Loss: 0.5006
Accuracy on the dataset: 85.156%
